# Lars

In [72]:
import numpy as np
import pandas as pd
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

In [73]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [74]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, 7),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [75]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=30, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=7, bias=True)
  )
)


## Daten

In [76]:
df = pd.read_csv("data/train.csv")

In [77]:
df

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II
20754,20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
20755,20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II
20756,20756,Male,33.852953,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,2.144838,no,0.000000,0.973834,no,Automobile,Overweight_Level_II


In [78]:
df_ohe = pd.get_dummies(df[df.columns.values[:-1]]) # OHE without target column

In [79]:
df_ohe = df_ohe.join(df["NObeyesdad"], how="left")

In [80]:
# Encode target
class_vals = sorted(df_ohe["NObeyesdad"].unique())
class_to_label = {cls: idx for idx, cls in enumerate(class_vals)}

In [81]:
X = torch.tensor(df_ohe.drop(columns=["NObeyesdad", "id"]).astype(float).values
, dtype=torch.float)
y = torch.tensor([class_to_label[class_] for class_ in df_ohe["NObeyesdad"]], dtype=torch.float)

In [82]:
print(X.shape, y.shape)

torch.Size([20758, 30]) torch.Size([20758])


In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible


### Untrained Predictions

In [84]:
X_test.shape

torch.Size([4152, 30])

In [85]:
model(X_test.to(device))

tensor([[-6.5511, -3.4616, -3.3184,  ..., -1.7497, -2.4608, -0.8775],
        [-4.4415, -2.3455, -2.0968,  ..., -1.2467, -1.5368, -0.7757],
        [-4.8261, -2.4303, -2.3638,  ..., -1.4630, -1.9263, -1.1589],
        ...,
        [-4.5780, -2.4747, -2.2483,  ..., -1.2041, -1.9235, -1.0877],
        [-4.9274, -2.4795, -2.4625,  ..., -1.4855, -2.1456, -1.6364],
        [-4.5717, -2.3628, -2.1972,  ..., -1.2661, -1.6509, -0.8316]],
       grad_fn=<AddmmBackward0>)

## Training auf Daten

### Loss und Optimizer

In [86]:
loss_fn = nn.CrossEntropyLoss()

In [87]:
optimizer = torch.optim.SGD(params=model.parameters(), 
                            lr=0.1)

In [88]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

### Training

In [89]:
y_logits = model(X_train.to(device))[:1]
y_logits

tensor([[-5.5310, -2.9015, -2.7354, -6.3878, -1.3955, -1.7333, -0.5911]],
       grad_fn=<SliceBackward0>)

In [90]:
y_pred_prob = torch.sigmoid(y_logits)
y_pred_prob

tensor([[0.0039, 0.0521, 0.0609, 0.0017, 0.1985, 0.1502, 0.3564]],
       grad_fn=<SigmoidBackward0>)

In [91]:
y_pred = torch.round(y_pred_prob)

print(y_test[:5])
print(y_pred[:5])

tensor([4., 5., 2., 3., 0.])
tensor([[0., 0., 0., 0., 0., 0., 0.]], grad_fn=<SliceBackward0>)


In [92]:
y_logits   

tensor([[-5.5310, -2.9015, -2.7354, -6.3878, -1.3955, -1.7333, -0.5911]],
       grad_fn=<SliceBackward0>)

In [93]:
y_train

tensor([2., 0., 3.,  ..., 5., 4., 2.])

In [94]:
torch.manual_seed(42)

# Set the number of epochs
epochs = 100

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device).long()
X_test, y_test = X_test.to(device), y_test.to(device).long()

# Build training and evaluation loop
for epoch in range(epochs):
    ### Training
    model.train()

    # 1. Forward pass (model outputs raw logits)
    y_logits = model(X_train)  # no need to squeeze
    y_pred = torch.argmax(y_logits, dim=1)  # get predicted class

    # 2. Calculate loss/accuracy
    loss = loss_fn(y_logits, y_train)  # no sigmoid needed for CrossEntropyLoss
    acc = accuracy_fn(y_true=y_train, y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model(X_test)
        test_pred = torch.argmax(test_logits, dim=1)
        # 2. Calculate loss/accuracy
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test, y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")


Epoch: 0 | Loss: 2.82792, Accuracy: 7.28% | Test loss: 21.97626, Test acc: 15.82%
Epoch: 10 | Loss: 1.94914, Accuracy: 14.33% | Test loss: 1.94974, Test acc: 13.13%
Epoch: 20 | Loss: 1.93990, Accuracy: 13.27% | Test loss: 2.00784, Test acc: 12.38%
Epoch: 30 | Loss: 1.94058, Accuracy: 19.66% | Test loss: 1.94181, Test acc: 19.41%
Epoch: 40 | Loss: 1.93783, Accuracy: 19.61% | Test loss: 1.93919, Test acc: 19.39%
Epoch: 50 | Loss: 1.93577, Accuracy: 19.60% | Test loss: 1.93723, Test acc: 19.36%
Epoch: 60 | Loss: 1.93424, Accuracy: 19.60% | Test loss: 1.93578, Test acc: 19.36%
Epoch: 70 | Loss: 1.93310, Accuracy: 19.60% | Test loss: 1.93470, Test acc: 19.36%
Epoch: 80 | Loss: 1.93224, Accuracy: 19.60% | Test loss: 1.93391, Test acc: 19.36%
Epoch: 90 | Loss: 1.93158, Accuracy: 19.60% | Test loss: 1.93330, Test acc: 19.36%
